In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean

from numpy.random import seed
seed(7)

from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [2]:
df = pd.read_csv("processminer-rare-event-mts - data.csv") 
df.head(n=5)  # visualize the data.

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
0,5/1/99 0:00,0,0.376665,-4.596435,-4.095756,13.497687,-0.118830,-20.669883,0.000732,-0.061114,...,10.091721,0.053279,-4.936434,-24.590146,18.515436,3.473400,0.033444,0.953219,0.006076,0
1,5/1/99 0:02,0,0.475720,-4.542502,-4.018359,16.230659,-0.128733,-18.758079,0.000732,-0.061114,...,10.095871,0.062801,-4.937179,-32.413266,22.760065,2.682933,0.033536,1.090502,0.006083,0
2,5/1/99 0:04,0,0.363848,-4.681394,-4.353147,14.127997,-0.138636,-17.836632,0.010803,-0.061114,...,10.100265,0.072322,-4.937924,-34.183774,27.004663,3.537487,0.033629,1.840540,0.006090,0
3,5/1/99 0:06,0,0.301590,-4.758934,-4.023612,13.161566,-0.148142,-18.517601,0.002075,-0.061114,...,10.104660,0.081600,-4.938669,-35.954281,21.672449,3.986095,0.033721,2.554880,0.006097,0
4,5/1/99 0:08,0,0.265578,-4.749928,-4.333150,15.267340,-0.155314,-17.505913,0.000732,-0.061114,...,10.109054,0.091121,-4.939414,-37.724789,21.907251,3.601573,0.033777,1.410494,0.006105,0


In [3]:
#select type 96
df = df.loc[(df['x28'] == 82)]
df

,time,y,x1,x2,x3,x4,x5,x6,x7,x8,...,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61
2971,5/5/99 18:32,1,0.360185,-2.644952,-6.353075,-205.692770,-0.189097,-60.424095,-0.009338,-0.051043,...,8.229416,1.749324,0.566952,38.421451,12.474237,2.350292,0.023598,0.516253,0.002627,0
2972,5/5/99 18:50,0,0.201489,-0.060164,-6.503206,-201.722830,-0.784113,-59.001320,0.088880,0.008773,...,8.229416,1.710262,0.567667,41.340885,12.820184,2.557811,0.019314,1.175036,0.002643,0
2973,5/5/99 18:52,0,0.255406,-0.056882,-4.580849,-203.499140,-0.865595,-60.499168,0.120670,0.038986,...,8.229416,1.705867,0.567747,42.166080,14.299585,2.597484,0.019173,-0.098814,0.002645,0
2974,5/5/99 18:54,0,0.056323,0.053060,-4.258417,-201.103230,-0.947092,-62.059639,0.130069,0.038986,...,8.229416,1.701473,0.567827,49.618717,12.321130,2.222118,0.019032,-0.371641,0.002647,0
2975,5/5/99 18:56,0,0.183178,0.283271,-4.216759,-201.523030,-1.028574,-56.403343,0.121341,0.038986,...,8.229416,1.697078,0.567906,57.666080,21.467554,2.429637,0.018891,0.109125,0.002649,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15080,5/24/99 2:20,0,0.179074,-1.007458,-5.333533,188.887152,0.908971,-12.681999,0.010803,-0.061114,...,-3.100418,0.839900,-5.034651,-82.681698,13.018640,-2.047413,0.008882,0.539767,0.002039,0
15081,5/24/99 2:22,0,0.191113,-0.814391,-5.495205,187.161719,0.919897,-12.438621,0.010803,-0.061114,...,-3.096024,0.844783,-5.029620,-72.294247,9.411371,-2.111499,0.008784,1.241793,0.002045,0
15082,5/24/99 2:24,0,0.227972,-0.779841,-5.719491,186.659857,0.930807,-13.293144,0.010803,-0.061114,...,-3.091629,0.849910,-5.024590,-77.832821,9.200464,-2.721852,0.008686,0.390879,0.002050,0
15083,5/24/99 2:26,0,0.182320,-0.663649,-5.789733,187.856268,0.941732,-13.007774,0.010803,-0.061114,...,-3.087235,0.855037,-5.019559,-76.636166,10.464258,-2.111499,0.008588,-0.081922,0.002056,0


In [4]:
sign = lambda x: (1, -1)[x < 0]

def curve_shift(df, shift_by):
    '''
    This function will shift the binary labels in a dataframe.
    The curve shift will be with respect to the 1s. 
    For example, if shift is -2, the following process
    will happen: if row n is labeled as 1, then
    - Make row (n+shift_by):(n+shift_by-1) = 1.
    - Remove row n.
    i.e. the labels will be shifted up to 2 rows up.
    
    Inputs:
    df       A pandas dataframe with a binary labeled column. 
             This labeled column should be named as 'y'.
    shift_by An integer denoting the number of rows to shift.
    
    Output
    df       A dataframe with the binary labels shifted by shift.
    '''

    vector = df['y'].copy()
    for s in range(abs(shift_by)):
        tmp = vector.shift(sign(shift_by))
        tmp = tmp.fillna(0)
        vector += tmp
    labelcol = 'y'
    # Add vector to the df
    df.insert(loc=0, column=labelcol+'tmp', value=vector)
    # Remove the rows with labelcol == 1.
    df = df.drop(df[df[labelcol] == 1].index)
    # Drop labelcol and rename the tmp col as labelcol
    df = df.drop(labelcol, axis=1)
    df = df.rename(columns={labelcol+'tmp': labelcol})
    # Make the labelcol binary
    df.loc[df[labelcol] > 0, labelcol] = 1

    return df

In [5]:
'''
Shift the data by 2 units, equal to 4 minutes.

Test: Testing whether the shift happened correctly.
'''
print('Before shifting')  # Positive labeled rows before shifting.
one_indexes = df.index[df['y'] == 1]
display(df.iloc[(one_indexes[0]-3):(one_indexes[0]+2), 0:5].head(n=5))

# Shift the response column y by 2 rows to do a 4-min ahead prediction.
df = curve_shift(df, shift_by = -2)

print('After shifting')  # Validating if the shift happened correctly.
display(df.iloc[(one_indexes[0]-4):(one_indexes[0]+1), 0:5].head(n=5))  

Before shifting


,time,y,x1,x2,x3
13104,5/21/99 5:58,0,0.259967,3.597535,-4.503653
13105,5/21/99 6:00,0,0.232211,3.552007,-4.323244
13106,5/21/99 6:02,0,0.190124,3.730132,-4.080037
13107,5/21/99 6:04,0,0.271485,3.806531,-4.116658
13108,5/21/99 6:06,0,0.202375,4.056804,-3.836913


After shifting


,y,time,x1,x2,x3
13117,0.0,5/21/99 6:24,0.428140,5.344095,-3.820789
13118,0.0,5/21/99 6:26,0.430785,5.544866,-3.467225
13119,0.0,5/21/99 6:28,0.465425,5.709054,-3.491588
13120,0.0,5/21/99 6:30,0.428310,5.801271,-3.477131
13121,0.0,5/21/99 6:32,0.354082,5.968555,-3.769824


In [6]:
# Remove time & categorical column
df = df.drop(['time','x28','x61'], axis=1)

In [7]:
X = df.drop(['y'], axis=1)
y = df['y']

In [8]:
X
X=X.fillna(0)
X

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60
2972,0.201489,-0.060164,-6.503206,-201.722830,-0.784113,-59.001320,0.088880,0.008773,0.039370,0.051841,...,29.984624,8.229416,1.710262,0.567667,41.340885,12.820184,2.557811,0.019314,1.175036,0.002643
2973,0.255406,-0.056882,-4.580849,-203.499140,-0.865595,-60.499168,0.120670,0.038986,0.009922,0.021628,...,29.984624,8.229416,1.705867,0.567747,42.166080,14.299585,2.597484,0.019173,-0.098814,0.002645
2974,0.056323,0.053060,-4.258417,-201.103230,-0.947092,-62.059639,0.130069,0.038986,-0.010220,0.001791,...,29.984624,8.229416,1.701473,0.567827,49.618717,12.321130,2.222118,0.019032,-0.371641,0.002647
2975,0.183178,0.283271,-4.216759,-201.523030,-1.028574,-56.403343,0.121341,0.038986,-0.019986,0.011557,...,29.984624,8.229416,1.697078,0.567906,57.666080,21.467554,2.429637,0.018891,0.109125,0.002649
2976,0.122141,0.353668,-3.987717,-203.900150,-1.110064,-63.053932,0.120670,0.038986,-0.030057,0.001791,...,29.984624,8.229416,1.692684,0.567986,62.277408,10.818994,1.916942,0.018712,0.783892,0.002650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15080,0.179074,-1.007458,-5.333533,188.887152,0.908971,-12.681999,0.010803,-0.061114,0.311597,0.382788,...,29.984624,-3.100418,0.839900,-5.034651,-82.681698,13.018640,-2.047413,0.008882,0.539767,0.002039
15081,0.191113,-0.814391,-5.495205,187.161719,0.919897,-12.438621,0.010803,-0.061114,0.299848,0.371675,...,29.984624,-3.096024,0.844783,-5.029620,-72.294247,9.411371,-2.111499,0.008784,1.241793,0.002045
15082,0.227972,-0.779841,-5.719491,186.659857,0.930807,-13.293144,0.010803,-0.061114,0.289777,0.361604,...,29.984624,-3.091629,0.849910,-5.024590,-77.832821,9.200464,-2.721852,0.008686,0.390879,0.002050
15083,0.182320,-0.663649,-5.789733,187.856268,0.941732,-13.007774,0.010803,-0.061114,0.280011,0.342711,...,29.984624,-3.087235,0.855037,-5.019559,-76.636166,10.464258,-2.111499,0.008588,-0.081922,0.002056


In [9]:
train_df = df.iloc[:3502]
test_df = df.iloc[-876:]

X_train = train_df.drop(['y'], axis=1)
X_train=X_train.fillna(0)
y_train = train_df['y']
y_train=y_train.fillna(0)

X_test = test_df.drop(['y'], axis=1)
X_test=X_test.fillna(0)
y_test = test_df['y']
y_test=y_test.fillna(0)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 2000)
#X_train.shape, X_test.shape

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


(3502, 59)
(3502,)
(876, 59)
(876,)


In [10]:
# Feature scaling

In [11]:
cols = X_train.columns

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns=[cols])
X_test = pd.DataFrame(X_test, columns=[cols])


In [12]:
# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.98416


In [12]:
from xgboost import XGBClassifier
model = XGBClassifier()

clf_0 = model.fit(X_train, y_train)


In [13]:
pred_y_0 = clf_0.predict(X_test)

print(accuracy_score(pred_y_0, y_test))

0.9726027397260274


In [14]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_0, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_0)

              precision    recall  f1-score   support

     class 0       1.00      0.98      0.99       872
     class 1       0.00      0.00      0.00         4

    accuracy                           0.97       876
   macro avg       0.50      0.49      0.49       876
weighted avg       0.99      0.97      0.98       876



array([[852,  20],
       [  4,   0]])

# Weighted XGBoost for Class Imbalance


In [ ]:
# define model
#scale_pos_weight=total_negative_examples / total_positive_examples
#total number of examples in the majority class / total number of examples in the minority class.
#4360/18=243
model = XGBClassifier(scale_pos_weight=243)

# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

In [16]:
model = XGBClassifier(scale_pos_weight=243)

clf_1 = model.fit(X_train, y_train)
pred_y_1 = clf_1.predict(X_test)

print(accuracy_score(pred_y_1, y_test))

0.9668949771689498


In [17]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_1, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_1)

              precision    recall  f1-score   support

     class 0       1.00      0.97      0.98       872
     class 1       0.00      0.00      0.00         4

    accuracy                           0.97       876
   macro avg       0.50      0.49      0.49       876
weighted avg       0.99      0.97      0.98       876



array([[847,  25],
       [  4,   0]])

# Tune with GridSearch CV

In [19]:
# define grid
weights = [1, 10, 25, 50, 75, 99, 100,700,800,1000,2000]
param_grid = dict(scale_pos_weight=weights)

In [20]:
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')

In [ ]:
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [51]:
# define model
model = XGBClassifier(scale_pos_weight=99)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance

print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.98417


In [18]:
model = XGBClassifier(scale_pos_weight=99)

clf_2 = model.fit(X_train, y_train)
pred_y_2 = clf_2.predict(X_test)

print(accuracy_score(pred_y_2, y_test))

0.9897260273972602


In [19]:
target_names = ['class 0', 'class 1']
print(classification_report(y_test, pred_y_2, target_names=target_names))

# View confusion matrix for test data and predictions
confusion_matrix(y_test, pred_y_2)

              precision    recall  f1-score   support

     class 0       1.00      0.99      0.99       872
     class 1       0.00      0.00      0.00         4

    accuracy                           0.99       876
   macro avg       0.50      0.50      0.50       876
weighted avg       0.99      0.99      0.99       876



array([[867,   5],
       [  4,   0]])

In [24]:
pred_y_2

array([0, 0, 0, ..., 1, 1, 0])

In [25]:
y_test

14709    0.0
14710    0.0
14711    0.0
14712    0.0
14713    0.0
        ... 
18393    0.0
18394    0.0
18395    0.0
18396    0.0
18397    0.0
Name: y, Length: 3655, dtype: float64